In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objects as go
from IPython.display import Image
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [15]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [28]:
df = pd.read_csv('lift2.csv')
products = pd.read_csv('instacart/products.csv')
aisles = pd.read_csv('instacart/aisles.csv')
departments = pd.read_csv('instacart/departments.csv')

In [29]:
product_info=products.merge(aisles,on='aisle_id').merge(departments,on='department_id')
product_info.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,78,Nutter Butter Cookie Bites Go-Pak,61,19,cookies cakes,snacks
2,102,Danish Butter Cookies,61,19,cookies cakes,snacks
3,172,Gluten Free All Natural Chocolate Chip Cookies,61,19,cookies cakes,snacks
4,285,Mini Nilla Wafers Munch Pack,61,19,cookies cakes,snacks


In [30]:
df=df[df['Clength']==1]

In [31]:
df=pd.merge(df[['antecedents','consequents','lift']], product_info[['product_name','department']], left_on=  ['antecedents'],
                   right_on= ['product_name'], 
                   how = 'left')
df['Adept']=df['department']
df=pd.merge(df[['antecedents','consequents','lift','Adept']], product_info[['product_name','department']], left_on=  ['consequents'],
                   right_on= ['product_name'], 
                   how = 'left')
df['Cdept']=df['department']
df=df[['antecedents','consequents','lift','Adept','Cdept']]

In [33]:
df.to_csv('SankeyforDept.csv')

In [22]:
df=df.groupby(by=['Adept','Cdept']).mean()

In [24]:
cfd=df.copy().reset_index()
# cfd.reset_index()

In [27]:
""" Interactive selection of the top_n consequents for an antecedent displayed in a Sankey graph."""
    
# Hide pandas chaining warning (not required)
pd.options.mode.chained_assignment = None
    
#Filter data by consequent and top number
x = cfd.sort_values(by=['lift'], ascending=False)
x = x[['Adept','Cdept','lift']]
#x = x.head(top_n)
    
# Create generic columns
column_names = ['X','Y','Z']
x.columns = column_names

# Unique list of antecedent/consequent names (nodes) for confidence ######
ant_names = set(x['X'].tolist())
cons_names = set(x['Y'].tolist())
nodes = list(set(list(ant_names) + list(cons_names)))

# Map nodes to values
name_map = { item : index for index, item in enumerate(nodes) }
x.loc[:,'X_ID'] = x["X"].map(name_map)
x.loc[:,'Y_ID'] = x["Y"].map(name_map)

# Generate the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 30,
      thickness = 20,
      line = dict(color = "black", width = .2),
      label = nodes,
      #color = colors
    ),
    link = dict(
      source = x['X_ID'].tolist(), # indices correspond to consequent labels
      target = x['Y_ID'].tolist(), # indices antecedent to consequent labels
      value = x['Z'].tolist() # Confidences
  ))])    

# Graph title
# title_text = 'aa'

# Update graph
fig.update_layout(
                 bargap=1,
                 boxgap=1,
                 funnelgap=1,
                 width=700,
                 font=dict(size=9),
                 titlefont=dict(size=20),
                 showlegend=True
                 )

# Display graph
fig.show()

